## Drop bad trials from covariates and images

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper')
sns.set_style('ticks')

/usr/local/python-2.7.6/lib/python2.7/site-packages/matplotlib-1.5.1-py2.7-linux-x86_64.egg/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/python-2.7.6/lib/python2.7/site-packages/matplotlib-1.5.1-py2.7-linux-x86_64.egg/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join, exists
from os import remove
import json

###Set directory and session information

In [3]:
directory = '/tier2/freeman/Nick/lfov.calibration'

In [4]:
key = '2016-02-03'
name = 'anm-0307509'

### List trials to exlude from analysis

In [5]:
drop = {'trials': []} #range(0,31)

###Load covariates

In [6]:
path = join(directory, 'reprocessed', name, key, 'covariates')
print exists(path)

True


In [7]:
covariates = pd.read_csv(join(path, 'covariates.csv'))

Save list of dropped trials

In [8]:
with open(join(path, 'drop.json'), 'w') as outfile:
    json.dump(drop, outfile, indent=2)

###Process covariates

In [9]:
keep = np.ones(len(covariates), dtype='bool')
for trial in drop['trials']:
    keep[(covariates.number == trial).values] = False

In [10]:
delete = covariates.index[~keep]
covariates = covariates[keep]

###Overwrite covariates

In [11]:
covariates.to_csv(join(path, 'covariates.csv'))

###Delete volumes

In [12]:
imagepath = join(directory, 'reprocessed', name, key, 'images')
print exists(imagepath)

True


In [13]:
for index in delete:
    remove(join(imagepath, 'image-%05d.bin' % index))

### Update meta

In [14]:
summarypath = join(directory, 'reprocessed', name, key, 'summary')
with open(join(summarypath, 'meta.json')) as infile:
    meta = json.load(infile)
meta['shape'] = [len(covariates)] + meta['shape'][1:]

In [15]:
with open(join(summarypath, 'meta.json'), 'w') as outfile:
    json.dump(meta, outfile, indent=2)